# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load cities.csv
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.93,125.41,74.64,37,36,3.09,TL,1603065704
1,Vaitape,-16.52,-151.75,80.64,75,63,7.49,PF,1603065705
2,Mataura,-46.19,168.86,51.01,92,100,8.99,NZ,1603065705
3,Constantine,36.37,6.61,48.20,76,32,3.36,DZ,1603065705
4,Arklow,52.79,-6.14,50.00,95,88,1.99,IE,1603065705
...,...,...,...,...,...,...,...,...,...
562,Ishikawa,26.42,127.82,80.01,88,20,9.17,JP,1603065812
563,Melfi,41.00,15.65,44.60,87,75,3.36,IT,1603065813
564,Acaraú,-2.89,-40.12,79.12,79,64,14.81,BR,1603065763
565,Yangambi,0.81,24.43,70.21,98,71,1.25,CD,1603065663


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store Latitude and Longitude in Locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in Humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot heatmap
fig = gmaps.figure(center =(46.0,-5.0), zoom_level=1.5)
max_intensity = np.max(humidity)

# Create Heat Layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100,
                                point_radius=3)

# Add Layer
fig.add_layer(heat_layer)

#Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the data.
# 70 > Max Temperature < 80
# Wind speed < 10
# 0 Cloudiness
# Drop any rows that do not contain all the conditions 
narrow_df = weather_data.loc[(weather_data["Wind Speed"]<= 10) & (weather_data["Cloudiness"]==0) & 
                            (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()
narrow_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,Salalah,17.02,54.09,75.20,36,0,3.36,OM,1603065713
98,Tura,25.52,90.22,71.96,92,0,2.04,IN,1603065676
157,Grand Gaube,-20.01,57.66,73.99,70,0,5.99,MU,1603065735
206,Kulachi,31.93,70.46,71.96,19,0,6.53,PK,1603065745
257,La Colmena,-25.88,-56.82,77.72,60,0,4.23,PY,1603065755
442,Kalakamati,-20.59,27.31,74.71,27,0,7.94,BW,1603065790
485,Upington,-28.45,21.26,71.60,10,0,7.40,ZA,1603065798
566,Makkah al Mukarramah,21.43,39.83,78.84,17,0,6.78,SA,1603065563


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrow_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add Hotel Name column into DataFrame
hotel_df["Hotel Name"] = ""

# Display the hotel DataFrame with changes made
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Salalah,OM,17.02,54.09,
98,Tura,IN,25.52,90.22,
157,Grand Gaube,MU,-20.01,57.66,
206,Kulachi,PK,31.93,70.46,
257,La Colmena,PY,-25.88,-56.82,
442,Kalakamati,BW,-20.59,27.31,
485,Upington,ZA,-28.45,21.26,
566,Makkah al Mukarramah,SA,21.43,39.83,


In [7]:
# Set base_url and parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key":g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # obtain city name, lat, lng from dataframe
    city_name = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # parameters 
    params["location"] = f"{lat},{lng}"
    
    # create url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params = params).json()
    
    # extract results 
    results = response["results"]
    
    # save the hotel name to dataframe
    try:
        print(f"Closest Hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
    print("---------------------------------------")
    
print("--------End of Search--------")

Retrieving Results for Index 43: Salalah.
Closest Hotel in Salalah is Al Jabal.
---------------------------------------
Retrieving Results for Index 98: Tura.
Closest Hotel in Tura is Hotel Polo Orchid.
---------------------------------------
Retrieving Results for Index 157: Grand Gaube.
Closest Hotel in Grand Gaube is LUX Grand Gaube Resort & Villas.
---------------------------------------
Retrieving Results for Index 206: Kulachi.
Closest Hotel in Kulachi is Stadium.
---------------------------------------
Retrieving Results for Index 257: La Colmena.
Closest Hotel in La Colmena is Hospedaje La Colmena.
---------------------------------------
Retrieving Results for Index 442: Kalakamati.
Missing field/result... skipping.
---------------------------------------
Retrieving Results for Index 485: Upington.
Closest Hotel in Upington is Protea Hotel by Marriott Upington.
---------------------------------------
Retrieving Results for Index 566: Makkah al Mukarramah.
Closest Hotel in Makka

In [9]:
# Display hotel dataframe with Hotel Name column information
hotel_df

,City,Country,Lat,Lng,Hotel Name
43,Salalah,OM,17.02,54.09,Al Jabal
98,Tura,IN,25.52,90.22,Hotel Polo Orchid
157,Grand Gaube,MU,-20.01,57.66,LUX Grand Gaube Resort & Villas
206,Kulachi,PK,31.93,70.46,Stadium
257,La Colmena,PY,-25.88,-56.82,Hospedaje La Colmena
442,Kalakamati,BW,-20.59,27.31,
485,Upington,ZA,-28.45,21.26,Protea Hotel by Marriott Upington
566,Makkah al Mukarramah,SA,21.43,39.83,AL kISWAH Towers Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))